In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
import weaviate
from llama_index import Document, ServiceContext
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompt_type import PromptType
from llama_index import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores import WeaviateVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.llms import OpenAI
from llama_index.llms import LangChainLLM

/Users/bdsaglam/knowledge/bellek/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:219: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.1.1/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/bdsaglam/knowledge/bellek/.venv/lib/python3.10/site-packages/litellm/proxy/_types.py:83: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.1.1/migration/
  extra = Extra.allow  # Allow extra fields
/Users/bdsaglam/knowledge/bellek/.venv/lib/python3.10/site-packages/litellm/proxy/_types.py:86: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 st

In [3]:
df = pd.read_json('/Users/bdsaglam/knowledge/bellek/data/generated/musique-kg-llm/train/dataset.jsonl', orient='records', lines=True)

In [4]:
def make_docs(example, only_supporting=False):
    ps = example["paragraphs"]
    for p in ps:
        if only_supporting and not p["is_supporting"]:
            continue
        idx = p["idx"]
        title = p["title"]
        body = p["paragraph_text"]
        is_supporting = p["is_supporting"]
        text = f"# {title}\n{body}"
        yield Document(
            text=text,
            metadata={"parent_id": example["id"], "idx": idx, "is_supporting": is_supporting},
            excluded_llm_metadata_keys=["parent_id", "idx", "is_supporting"],
        )

In [5]:
weaviate_client = weaviate.Client("http://localhost:8080")
vector_store = WeaviateVectorStore(weaviate_client=weaviate_client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [6]:
llm = OpenAI(model='gpt-3.5-turbo', temperature=0)
embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

/Users/bdsaglam/knowledge/bellek/.venv/lib/python3.10/site-packages/huggingface_hub/inference/_text_generation.py:121: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.1.1/migration/
  @validator("best_of")
/Users/bdsaglam/knowledge/bellek/.venv/lib/python3.10/site-packages/huggingface_hub/inference/_text_generation.py:140: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.1.1/migration/
  @validator("repetition_penalty")
/Users/bdsaglam/knowledge/bellek/.venv/lib/py

In [7]:
def make_query_engine(example):
    documents = list(make_docs(example, only_supporting=False))
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
    system_prompt = (
        "You are a helpful assistant that answers the question about the given text in 2-4 words."
    )
    query_engine = index.as_query_engine(
        service_context=service_context,
        vector_store_query_mode="hybrid",
        alpha=0.7,
        similarity_top_k=3,
        system_prompt=system_prompt,
    )
    return query_engine

In [8]:
def answer(example):
    query_engine = make_query_engine(example)
    return query_engine.query(example['question']).response

In [9]:
df['predicted_answer'] = df.apply(answer, axis=1)
df.to_json('/Users/bdsaglam/knowledge/bellek/data/generated/musique-kg-llm/train/baseline-weaviate.jsonl', orient='records', lines=True)

In [10]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def fuzzy_match(a, b, threshold=0.7):
    return similarity(a, b) >= threshold

def is_correct(example):
    return (example["answer"] in example["predicted_answer"]) or (
        fuzzy_match(example["predicted_answer"], example["answer"])
    )

In [11]:
df["is_correct"] = df.apply(is_correct, axis=1)
df["is_correct"].mean()

0.17

In [12]:
df[['id', 'question', 'answer', 'predicted_answer', 'is_correct']]

,id,question,answer,predicted_answer,is_correct
0,2hop__128801_205185,What county is the town where KNFM is licensed...,Midland County,The town where KNFM is licensed is not the cap...,False
1,2hop__719559_217649,What's the record label of the artist who put ...,Warner Bros.,"The record label of the artist who put out ""Me...",False
2,2hop__128806_205185,What region is the town where KQRX is liscense...,Midland County,The town where KQRX is licensed is not mention...,False
3,2hop__837090_278127,What is the record label of the Do It Again pe...,Roc-A-Fella Records,"The record label of the performer of ""Do It Ag...",False
4,2hop__128895_11424,How many households were there in the town WPU...,"15,504",The number of households in the town WPUR is l...,False
...,...,...,...,...,...
95,2hop__651488_94210,Who was the place where Pieta is located desig...,Giorgio Vasari,Le Corbusier,False
96,2hop__362083_467995,What is the record label of the performer of M...,RCA Records,"The record label of the performer of ""Make the...",False
97,2hop__525596_543261,The Roman Catholic Diocese of Jim Norton's bir...,Delaware,"I'm sorry, but I couldn't find any information...",False
98,2hop__394596_8607,What metro area is JAKAZiD's birthplace a part...,South Hampshire,The birthplace of JAKAZiD is part of the Ports...,False
